In [12]:
import sys
import os

import warnings

import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.data import Dataset
from PIL import Image
from image import *
import tensorflow.image
from keras.optimizers import SGD, Adam
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Conv2D 

import numpy as np
import json
import cv2
import dataset
import time


In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

tf.enable_eager_execution()

In [14]:
train_list = []
with open('train_data_list.txt', 'r') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        train_list.append(inner_list)
train_list = [val.replace('\'','') for val in train_list[0]]

In [15]:
test_list = []
with open('test_data_list.txt', 'r') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        test_list.append(inner_list)
test_list = [val.replace('\'','') for val in test_list[0]]

In [39]:
# import image
def load_data(paths, train = True):
    for img_path in paths:
        gt_path = img_path.decode("utf-8").replace('.jpg','.h5').replace('images','ground-truth')
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels = 3)
        img = tf.cast(img, tf.float32)
        img = (img/127.5) - 1 # normalizing the images to [-1, 1]
        
        gt_file = h5py.File(gt_path, 'r')
        target = np.asarray(gt_file['density'])

        target = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64

    
        yield (img, target)

In [40]:
train_dataset = tf.data.Dataset.from_generator(
                        load_data, args = [train_list] , output_types = (tf.float32, tf.float32), output_shapes = ((None,None,3), (None,None)))
train_dataset = train_dataset.shuffle(100000)


In [42]:
test_dataset = tf.data.Dataset.from_generator(
                   load_data, args = [test_list] , output_types = (tf.float32, tf.float32), output_shapes = ((None,None,3), (None,None)))

# Define function

In [43]:
batch_size = 100
epochs = 15
IMG_HEIGHT = 255
IMG_WIDTH = 255
learning_rate = 0.01

In [44]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(None, None,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(1))
    return model

In [45]:
model = create_model()

In [46]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [75]:
def loss_fn(model, input_image, gt_image):

    output = model(np.expand_dims(input_image,0), training=True)
    output = tf.squeeze(output, [0,3])
    # mean squared error
    loss = tf.reduce_mean(tf.square(output - gt_image))
    return loss

In [72]:
def grad(model, input_image, gt_image):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, input_image, gt_image)
    return tape.gradient(loss, model.trainable_weights)

In [80]:
for step, (images, gt_images) in enumerate(train_dataset.take(24)):
    output = model(tf.expand_dims(images,0))
    output = tf.squeeze(output, [0,3])
    print(images.shape)
    print(output.shape, '######', gt_images.shape)

(768, 1024, 3)
(96, 128) ###### (96, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(388, 576, 3)
(49, 72) ###### (48, 72)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(681, 1024, 3)
(86, 128) ###### (85, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(683, 1024, 3)
(86, 128) ###### (85, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(683, 1024, 3)
(86, 128) ###### (85, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(387, 580, 3)
(49, 73) ###### (48, 72)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(649, 1024, 3)
(82, 128) ###### (81, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(410, 563, 3)
(52, 71) ###### (51, 70)
(662, 962, 3)
(83, 121) ###### (82, 120)
(576, 1024, 3)
(72, 128) ###### (72, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(768, 1024, 3)
(96, 128) ###### (96, 128)
(480, 722, 3)
(60, 91) ###### (60, 90)


In [76]:
# train my model
print('Learning started. It takes sometime.')

for epoch in range(3):
    avg_loss = 0.
    train_step = 0
    test_step = 0
    test_mae = 0
    
    # train process
    for step, (images, gt_images) in enumerate(train_dataset):
        
        grads = grad(model, images, gt_images)                
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        loss = loss_fn(model, images, gt_images)
        avg_loss += loss
        train_step += 1
     
    avg_loss = avg_loss / train_step
    
    # test process
    for step, (images, gt_images) in enumerate(test_dataset): 
        
        output = model(np.expand_dims(images,0))
        test_step += 1    
        
        mae += abs(np.sum(output)-np.sum(gt_images))
        
    mae = mae / test_step  
   

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'Test MAE = ', '{:.4f}'.format(mae))
    
print('Learning Finished!')

Learning started. It takes sometime.


InvalidArgumentError: Incompatible shapes: [49,72] vs. [48,72] [Op:Sub] name: sub/

In [ ]:
def generate_images(model, test_input, tar):
  # the training=True is intentional here since
  # we want the batch statistics while running the model
  # on the test dataset. If we use training=False, we will get
  # the accumulated statistics learned from the training dataset
  # (which we don't want)
    prediction = model(test_input, training=True)
    plt.figure(figsize=(15,15))

    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
from matplotlib import cm as c
img = transform(Image.open('part_A/test_data/images/IMG_100.jpg').convert('RGB')).cuda()

output = model(img.unsqueeze(0))
print("Predicted Count : ",int(output.detach().cpu().sum().numpy()))
temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
plt.imshow(temp,cmap = c.jet)
plt.show()
temp = h5py.File('part_A/test_data/ground-truth/IMG_100.h5', 'r')
temp_1 = np.asarray(temp['density'])
plt.imshow(temp_1,cmap = c.jet)
print("Original Count : ",int(np.sum(temp_1)) + 1)
plt.show()
print("Original Image")
plt.imshow(plt.imread('part_A/test_data/images/IMG_100.jpg'))
plt.show()